1. The Pilots Brothers'Refrigerator [Acwing116](https://www.acwing.com/problem/content/118/)

* 题解
  * 把每一步操作抽象成一棵树，进行遍历剪枝。
  * 用一个16位的二进制数来表示矩阵


In [ ]:
import copy
num = 0
for i in range(4):
    _str = input().strip()
    for s in _str:
        num <<= 1
        num = num + 1 if s=='-' else num

trick = [-1]*16
vis = [False]*16
res = [100,None]

def change_num(num,pos):
    row = pos // 4    # 行
    col = pos % 4     # 列
    for i in range(4):
        num ^= (1<<(4*i+col))       # 对列取反
        num ^= (1<<(4*row+i))       # 对行取反
    num ^= (1<<pos)
    return num

def search(num,deepth,res):
    print(trick,bin(num))
    if num == 0:
        if deepth < res[0]:
            res[-1] = copy.deepcopy(trick)
            res[0] = deepth
        return None
    
    for i in range(15,-1,-1):
        if vis[i] == False:
            vis[i] = True
            trick[i] = deepth
            num = change_num(num,i)
            search(num,deepth+1,res)
            trick[i] = -1
            vis[i] = False

search(num,0,res)
print(res[0],res[-1])

2. 占卜DIY [Acwing](https://www.acwing.com/problem/content/119/)

In [ ]:
reflact = {
    'A':1,
    '0':10,
    'J':11,
    'Q':12,
    'K':13
}
puk = [0]*14
heap = [[] for _ in range(14)]
for i in range(1,14):
    temp = input().split()
    for x in temp:
        if x.isnumeric() and int(x) !=0:
            heap[i].append(int(x))
        else:
            heap[i].append(reflact[x])
for i in range(4):
    pk = heap[-1][i]        # 从最后一堆取出来一张牌
    heap[-1][i] = -1

    while pk != 13 and pk != -1:
        puk[pk] += 1        # 统计pk
        for j in range(3,-1,-1):        # 抽取新的pk
            if heap[pk][j] != -1:
                temp = heap[pk][j]
                heap[pk][j] = -1
                pk = temp
                break
            if j == 0:
                pk = -1
res = 0
for item in puk:
    if item == 4:
        res += 1
print(res)

3. 分形之城 [Acwing](https://www.acwing.com/problem/content/100/)

太难了，暂时不做

4. Raid [Acwing121](https://www.acwing.com/problem/content/121/)

求空间中的最近点对 (十分经典的题目)

* 题解（先考虑不分阵营的点）：
  * 每次把数据分成左右两部分，递归求出左右两部分的最小值 lmind 和 rmind
  * 合并左右两部分的距离
    1. 求出最短距离 mind = min(lmind,rmind)
    2. 在划分线的 [mid-mind,mid+mind] 之间找点
    3. 假设 [mid-mind,mid] 中任一点p[$x_p$,$y_p$]，那么在x轴范围[mid,mind]，y轴范围[$y_p$-mind,$y_p$+mind]这个矩形内部，最多有6个点与p计算一下距离
    4. 这里有个问题：对于p，我们怎么找到右边在这个矩形内部的点呢？实际在做的时候并不是按照这个思路去做。而是把 [mid-mind,mid+mind] 中所有的点按照y轴排序，然后设置两个指针，判断两个点之间的距离与mind的关系来剪枝实现。

用python写回超时

In [ ]:
# 使用数组来存储点 [x,y,type]
import math
x,y,t = 0,1,2
inf = math.inf

N = int(input())

# 距离函数
distance = lambda p0,p1: math.sqrt((p0[x]-p1[x])**2+(p0[y]-p1[y])**2) if p0[t] != p1[t] else math.inf

# dfs函数
def dfs(left,right,points,swap_array):        # 处理区间[left,right]，闭区间
    if left >= right:        # 如果点的个数 <= 1 返回 inf
        return inf
    mid = (left+right) // 2
    mid_x = points[mid][x]
    mind = min(dfs(left,mid,points,swap_array),dfs(mid+1,right,points,swap_array))
    """根据y轴对数据进行归并排序"""
    i,j = left,mid+1
    for k in range(left,right + 1):
        if j > right or (i <= mid and points[i][y] < points[j][y]):
            swap_array[k] = points[i]
            i += 1
        else:
            swap_array[k] = points[j]
            j+=1
    for k in range(left,right+1):
        points[k] = swap_array[k]
    
    """筛选出来x轴在[mid_x-mind,mid_x+mind]范围内的点"""
    temp = []
    for k in range(left,right+1):
        if points[k][x] < mid_x+mind and points[k][y] > mid_x-mind:
            temp.append(points[k])
    
    """遍历这些点，并进行剪枝"""
    for i in range(1,len(temp)):
        for j in range(i-1,-1,-1):      # 这里实现剪枝作用
            if temp[i][y]-temp[j][y] >= mind:
                break
            mind = min(mind,distance(temp[i],temp[j]))
    return mind

def work():
    """读入数据"""
    num = int(input())
    points = []
    for i in range(num):
        a,b = list(map(int,input().split()))
        points.append([a,b,0])
    for i in range(num):
        a,b = list(map(int,input().split()))
        points.append([a,b,1])
    swap_array = [None]*len(points)
    print('%.3f'%dfs(0,2*num-1,points,swap_array))

for _ in range(N):
    work()
    


5. 防线 [Acwing120](https://www.acwing.com/problem/content/122/)

* 题解：
  * 前缀和 + 二分
  * 不能分配一个 $2^31-1$大小的数组，会报错
  * 这里抓住最多只有一个奇数这个特点  **这一点刚开始没有想到**

In [ ]:
T = int(input())

def check(lines,index):
    res = 0
    for line in lines:
        if line[0] <= index:
            res += (min(line[1],index) - line[0])//line[2] + 1
    return res


for t in range(T):                  # 读数据
    N = int(input())
    E = 0
    S = 1<<31
    lines = []
    for _ in range(N):
        s,e,d = list(map(int,input().split()))
        E = max(e,E)
        S = min(s,S)
        lines.append([s,e,d])
    print(lines[S:E+1])
    
    # 使用后继的二分法
    l,r = S,E+1
    while l<r:
        mid = (l+r) >> 1
        if check(lines,mid) % 2 ==1:
            r = mid
        else:
            l = mid + 1
    
    if l > E:
        print("There's no weakness.")
    else:
        print(l,check(lines,l)-check(lines,l-1))


6. 赶牛入圈 [Acwing121](https://www.acwing.com/problem/content/123/)

* 考点
  * 二分
  * 前缀和
  * 离散化

7. 糖果传递

* 考点：
  * 排序
  * 中位数
  * 环形均分纸牌

8. Soldiers

* 考点
  * 排序
  * 中位数
  * 货仓选址问题扩展

